In [5]:
import requests
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,"lxml")
My_table = soup.find('table',{'class':'wikitable sortable'})
PostalCode = []
Borough = []
Neighbourhood = []
links = My_table.findAll('tr') 
for link in links:
    try:   
        td = link.find_all('td')
        #print(td[0].text,td[1].text,td[2].text)
        PostalCode.append(td[0].text.strip("\n"))
        Borough.append(td[1].text.strip("\n"))
        Neighbourhood.append(td[2].text.strip("\n"))
    except IndexError: 
        continue

import pandas as pd
df_data = {"PostalCode":PostalCode, "Borough":Borough,"Neighbourhood":Neighbourhood}
df = pd.DataFrame(df_data, columns=["PostalCode", "Borough", "Neighbourhood"])
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
postal_codes_df = pd.read_csv("Geospatial_Coordinates.csv")
postal_codes_df = postal_codes_df.rename(columns = {"Postal Code": "PostalCode"})
postal_codes_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
df_codes = pd.merge(df, postal_codes_df, on = "PostalCode")
df_codes.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
